<a href="https://colab.research.google.com/github/rg1235/Facial-Exp-recognition/blob/master/Material_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
from google.colab import drive
drive.mount('/content/gdrive')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9761693658645559828
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9263596078138559306
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17526313134634100068
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14800692839
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13238559270875896941
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.co

In [0]:
import cv2
import glob
import numpy as np

In [0]:
train=[]
train_labels=[]

In [0]:
#importing the image files for glass
files = glob.glob ("/content/gdrive/My Drive/FMD/image/glass/*.jpg")
for myFile in files:
    image = cv2.imread (myFile)
    train.append (image)
    train_labels.append([0])

In [0]:
#importing the image files for leather
files = glob.glob ("/content/gdrive/My Drive/FMD/image/leather/*.jpg") # your image path
for myFile in files:
    image = cv2.imread (myFile)
    train.append (image)
    train_labels.append([1])

In [0]:
#importing images for metal
files = glob.glob ("/content/gdrive/My Drive/FMD/image/metal/*.jpg") # your image path
for myFile in files:
    image = cv2.imread (myFile)
    train.append (image)
    train_labels.append([2])

In [0]:
#importing images for wood
files = glob.glob ("/content/gdrive/My Drive/FMD/image/wood/*.jpg") # your image path
for myFile in files:
    image = cv2.imread (myFile)
    train.append (image)
    train_labels.append([3])

In [0]:
from keras.utils import to_categorical
train_Y_one_hot = to_categorical(train_labels)

In [36]:
train_Y_one_hot

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [0]:
#converting to numpy array
train = np.array(train,dtype='float32') #as mnist
#train_labels = np.array(train_labels,dtype='float64') #as mnist

In [38]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_label,test_label = train_test_split(train, train_Y_one_hot, test_size=0.2, random_state=13)
print(train_X.shape,test_X.shape,train_label.shape,test_label.shape)

(320, 384, 512, 3) (80, 384, 512, 3) (320, 4) (80, 4)


In [39]:
train.shape

(400, 384, 512, 3)

In [0]:
from keras.applications import VGG16
 
vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(384, 512, 3))

Using a pretrained VGG16 model

In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,vertical_flip=True)
batch_size = 10
train_generator = datagen.flow(train_X,train_label,batch_size=batch_size)

In [0]:
nTrain=600
train_features = np.zeros(shape=(nTrain, 12, 16, 512))
train_labels_1 = np.zeros(shape=(nTrain,4))

In [0]:
i = 0
for inputs_batch, labels_batch in train_generator:
    #print(inputs_batch.shape)
    features_batch = vgg_conv.predict(inputs_batch)
    #print(features_batch.shape)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels_1[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break

In [0]:
train_features.shape
train_features = np.reshape(train_features, (nTrain, 12 * 16 * 512))

In [45]:
train_labels_1


array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [0]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D ,AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [0]:
from keras import models
from keras import layers
from keras import optimizers
 
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=12 * 16 * 512))
model.add(LeakyReLU(alpha=0.1))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               25166080  
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 1028      
Total params: 25,167,108
Trainable params: 25,167,108
Non-trainable params: 0
_________________________________________________________________


In [49]:
epochs=200
#model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(momentum=0.9),metrics=['accuracy'])
model.compile(optimizer=optimizers.RMSprop(lr=2e-4),loss='categorical_crossentropy',metrics=['acc'])
model_train = model.fit(train_features, train_labels_1, batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/200
600/600 [==============================] - 2s 3ms/step - loss: 3.0738 - acc: 0.4583
Epoch 2/200
600/600 [==============================] - 1s 2ms/step - loss: 1.1437 - acc: 0.6250
Epoch 3/200
600/600 [==============================] - 1s 2ms/step - loss: 0.9234 - acc: 0.6983
Epoch 4/200
600/600 [==============================] - 1s 2ms/step - loss: 0.7169 - acc: 0.7633
Epoch 5/200
600/600 [==============================] - 1s 2ms/step - loss: 0.4914 - acc: 0.8250
Epoch 6/200
600/600 [==============================] - 1s 2ms/step - loss: 0.4158 - acc: 0.8600
Epoch 7/200
600/600 [==============================] - 1s 2ms/step - loss: 0.3204 - acc: 0.8900
Epoch 8/200
600/600 [==============================] - 1s 2ms/step - loss: 0.2453 - acc: 0.9100
Epoch 9/200
600/600 [==============================] - 1s 2ms/step - loss: 0.2181 - acc: 0.9333
Epoch 10/200
600/600 [==============================] - 1s 2ms/step - loss: 0.1193 - acc: 0.9600
Epoch 11/200
600/600 [=================

In [0]:
#for testing we need to make the test features by passing it through the vgg16 model
nTest=80
test_features = vgg_conv.predict(test_X)
test_features = np.reshape(test_features, (80, 12 * 16 * 512))

In [51]:
test_eval = model.evaluate(test_features, test_label, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

Test loss: 3.223619079589844
Test accuracy: 0.8


In [0]:
model.save("model_softmax_(80).h5py")

In [53]:
from google.colab import files
files.download('model_softmax_(80).h5py')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 52766, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil